# Trabalho 02 de Métodos Quantitativos
###### Pedro Henrique Ventura 
###### Aluno de Pós Graduação (2019)
## Bibliotecas

In [1]:
#Obtenção dos dados do site:
import requests
from urllib import request, response, error, parse
from urllib.request import urlopen
from bs4 import BeautifulSoup

#Manipulação de dados, bases e geração de gráficos
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 1. Obtendo dados
#### Nesta etapa, obteve-se dados referêntes a quadrinhos na loja virtual da marvel (comicstore.marvel.com); Nome da HQ e Preço.
##### Obs: Caso não queira  baixar todos os dados, execute a partir do item 2

In [2]:
url = []
for i in range(1, 43): #Pegando todas as páginas que possuiam hqs para venda
    url.append('https://comicstore.marvel.com/comics-best-sellers?cu=0&bestsellerList_pg=' + str(i))

#Listas para armazenar nome, numeração e preço das hqs
hq_name = []
hq_numbering = []
hq_price = []

num = 0 #variável para contar de quais páginas já foram pegos 

for x in url: #pegando cada página
    num = num+1
    html = urlopen(x)
    soup = BeautifulSoup (html,"lxml")
    title = soup.title
    titleText = title.get_text()
    print("Página" + titleText + " - " + str(num) + " Obtida") #exibindo que já foi pego
    
    figcaption = soup.find_all('figcaption', class_='content-info') ## Obtendo o Título e Preço das obras
    for elemen in figcaption:
        wrappers = elemen.find('h5', class_='content-title cu-alc')
        hq_name.append(wrappers.get_text().replace('-)', ')'))

        wrappers = elemen.find('h6', class_='content-subtitle cu-alc')
        hq_numbering.append(wrappers.get_text().split("Issue ")[1])
        
        wrappers = elemen.find('h5', class_='item-price cu-alc')
        hq_price.append(float(wrappers.get_text().split("$")[1]))
        
        
        

PáginaComics Best Sellers - Marvel Comics - 1 Obtida


IndexError: list index out of range

In [ ]:
# Criando e salvando a base
data = {'Nome':hq_name, 'Numeacao': hq_numbering, 'Preco':hq_price} 
dsetMarvelHQ = pd.DataFrame(data) 
dsetMarvelHQ.to_csv(r'C:\Users\pe-ri\Documents\Python Scripts\PGCC-Métodos Quantitativos\PGCC-MQ\MQ02\dsetMarvelHQ.csv',index = None, header=True)


## 2. Visualizando a base gerada

In [ ]:
# Carregando a Base para não ter que ficar rodando sempre a parte 1
df = pd.read_csv(r'C:\Users\pe-ri\Documents\Python Scripts\PGCC-Métodos Quantitativos\PGCC-MQ\MQ02\dsetMarvelHQ.csv')

# Exibindo primeiros e últimos dados
print(df.head())
print(df.tail())

##### Com tais dados, optou-se por fazer uma análise com relação aos preços das HQs por se tratar de uma variável Discreta, e o nome ser uma V. Categórica

In [ ]:
# Exibindo informações estatíticas básicas dos dados 
print(df.describe())
boxplot = df.boxplot(column=['Preco'])
print(boxplot)

##### Foi possível perceber a existência de outliers por um simples boxplot. Logo, Fora feito então 2 processos, um com os outliers, e outro sem.

### 3. Com Outliers
#### 3.1 Calculando CDF

In [ ]:
"""Pegando a quantidade de ocorrências e calculando a probabilidade"""

#variáveis para armazenar total de valores distintos ocorrem ($totalOcorrencia), 
# e quais valores ocorrem e em quantas vezes ($ocorrencias)
totalOcorrencia = 0
ocorrencias = []
ocorrencias = dict()
#calculando quantidade de ocorrências

X = df['Preco'].sort_values(ascending=True) #ordenando min->max
totalAtributos = len(X) #pegando qdt total de atributos
    
for i in X: #agrupando e contando ocorrências
    try:
        ocorrencias[i] += 1
    except KeyError:
        ocorrencias[i] = 1
        totalOcorrencia = totalOcorrencia + 1

print("Total de ocorrências: ", totalOcorrencia)
print("Ocorrências: ", ocorrencias)


In [ ]:
#calculando a probabilidade de ocorrência
probabilidade = []
eixoX = []

for item, totalOcorrenciaIndividual in ocorrencias.items():
    probabilidade.append(totalOcorrenciaIndividual/totalAtributos)
    eixoX.append(item)
print("\nProbabilidade:", probabilidade)

#Calculando eixo X da CDF
ValorEixoX = []
ValorEixoX.append(probabilidade[0])

for i in range(1,totalOcorrencia):
        if i != 0:
            ValorEixoX.append(probabilidade[i]+ValorEixoX[i-1])


#### 3.2 Plotando CDF

In [ ]:
plt.xlabel('Preço da HQ')
plt.ylabel('Probabilidade')
plt.title('CDF Discreta')
plt.grid(True)
plt.step(eixoX, ValorEixoX, color='g')
plt.show()

#### 3.3 Calculando PMF e Criando tabela de relação do preço e PMF

In [ ]:
data = {'Nota':eixoX, 'PMF':probabilidade} 
pmf = pd.DataFrame(data) 

print("PMF por Preço")
print(pmf)

#### 3.4 Plotando PMF

In [ ]:
plt.xlabel('Preço da HQ')
plt.ylabel('Probabilidade')
plt.title('PMF')
plt.bar(eixoX, probabilidade,label='PMF')
plt.show()

### 4. Sem Outliers
#### 4.1 Removendo Outliers

In [ ]:
"""Refazendo mas removendo Outliers (>=25)"""
"""Outliers: 24.99: 10, 29.99: 5, 34.99: 1, 49.99: 1"""

outliers = [24.99, 29.99, 34.99, 49.99] #carregando outliers

for k in outliers:
    # Obtendo posição dos outliers na base
    indexNames = df[ df['Preco'] == k ].index
    # Removendo
    df.drop(indexNames , inplace=True)

#Exibindo estatus sem os outliers
print(df.describe())
boxplot = df.boxplot(column=['Preco'])

#### 4.2 Calculando CDF

In [ ]:
"""Pegando a quantidade de ocorrências e calculando a probabilidade"""

#variáveis para armazenar total de valores distintos ocorrem ($totalOcorrencia), 
# e quais valores ocorrem e em quantas vezes ($ocorrencias)
totalOcorrencia = 0
ocorrencias = []
ocorrencias = dict()
#calculando quantidade de ocorrências

X = df['Preco'].sort_values(ascending=True) #ordenando min->max
totalAtributos = len(X) #pegando qdt total de atributos
    
for i in X: #agrupando e contando ocorrências
    try:
        ocorrencias[i] += 1
    except KeyError:
        ocorrencias[i] = 1
        totalOcorrencia = totalOcorrencia + 1

print("Total de ocorrências: ", totalOcorrencia)
print("Ocorrências: ", ocorrencias)


In [ ]:
"""Pegando a quantidade de ocorrências e calculando a probabilidade"""

#variáveis para armazenar total de valores distintos ocorrem ($totalOcorrencia), e quais valores ocorrem e em quantas vezes ($ocorrencias)
totalOcorrencia = 0
ocorrencias = []
ocorrencias = dict()
#calculando quantidade de ocorrências

X = df['Preco'].sort_values(ascending=True) #ordenando min->max
totalAtributos = len(X) #pegando qdt total de atributos
    
for i in X: #agrupando e contando ocorrências
    try:
        ocorrencias[i] += 1
    except KeyError:
        ocorrencias[i] = 1
        totalOcorrencia = totalOcorrencia + 1

print("Total de ocorrências: ", totalOcorrencia)
print("Ocorrências: ", ocorrencias)


In [ ]:
#calculando a probabilidade de ocorrência
probabilidade = []
eixoX = []

for item, totalOcorrenciaIndividual in ocorrencias.items():
    probabilidade.append(totalOcorrenciaIndividual/totalAtributos)
    eixoX.append(item)
print("\nProbabilidade:", probabilidade)

#Calculando eixo X da CDF
ValorEixoX = []
ValorEixoX.append(probabilidade[0])

for i in range(1,totalOcorrencia):
        if i != 0:
            ValorEixoX.append(probabilidade[i]+ValorEixoX[i-1])


#### 4.3 Plotando CDF

In [ ]:
plt.xlabel('Preço da HQ')
plt.ylabel('Probabilidade')
plt.title('CDF Discreta')
plt.grid(True)
plt.step(eixoX, ValorEixoX, color='b')
plt.show()

#### 4.4 Calculando PMF e Criando tabela de relação do preço e PMF

In [ ]:
data = {'Nota':eixoX, 'PMF':probabilidade} 
pmf = pd.DataFrame(data) 

print("PMF por Preço")
print(pmf)

#### 4.5 Plotando PMF

In [ ]:
plt.xlabel('Preço da HQ')
plt.ylabel('Probabilidade')
plt.title('PMF')
plt.bar(eixoX, probabilidade,label='PMF')
plt.show()